##PySpark Window Functions

---

**PySpark Window functions are used to calculate results such as the rank, row number e.t.c over a range of input rows. In this article, I’ve explained the concept of window functions, syntax, and finally how to use them with PySpark SQL and PySpark DataFrame API. These come in handy when we need to make aggregate operations in a specific window frame on DataFrame columns.**

**When possible try to leverage standard library as they are little bit more compile-time safety, handles null and perform better when compared to UDF’s. If your application is critical on performance try to avoid using custom UDF at all costs as these are not guarantee on performance.**


---


##1. Window Functions


**PySpark Window functions operate on a group of rows (like frame, partition) and return a single value for every input row. PySpark SQL supports three kinds of window functions:**

- ranking functions

- analytic functions

- aggregate functions


---


**To perform an operation on a group first, we need to partition the data using Window.partitionBy() , and for row number and rank function we need to additionally order by on partition data using orderBy clause.**

In [0]:
simpleData = (
    ("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
)
columns = ["employee_name", "department", "salary"]

df = spark.createDataFrame(data=simpleData, schema=columns)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



##2. PySpark Window Ranking functions


###2.1 row_number Window Function


**row_number() window function is used to give the sequential row number starting from 1 to the result of each window partition.**

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
windowSpec = Window.partitionBy("department").orderBy("salary")


df. withColumn("row_number", row_number().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |2         |
|Robert       |Sales     |4100  |3         |
|Saif         |Sales     |4100  |4         |
|Michael      |Sales     |4600  |5         |
+-------------+----------+------+----------+



###2.2 rank Window Function


**rank() window function is used to provide a rank to the result within a window partition. This function leaves gaps in rank when there are ties.**

In [0]:
df.withColumn("rank", rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |1   |
|Scott        |Finance   |3300  |2   |
|Jen          |Finance   |3900  |3   |
|Kumar        |Marketing |2000  |1   |
|Jeff         |Marketing |3000  |2   |
|James        |Sales     |3000  |1   |
|James        |Sales     |3000  |1   |
|Robert       |Sales     |4100  |3   |
|Saif         |Sales     |4100  |3   |
|Michael      |Sales     |4600  |5   |
+-------------+----------+------+----+



###This is the same as the RANK function in SQL.


***
---



##2.3 dense_rank Window Function


**dense_rank() window function is used to get the result with rank of rows within a window partition without any gaps. This is similar to rank() function difference being rank function leaves gaps in rank when there are ties.**

In [0]:
""" dens_rank"""

df.withColumn("dense_rank", dense_rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |1         |
|Robert       |Sales     |4100  |2         |
|Saif         |Sales     |4100  |2         |
|Michael      |Sales     |4600  |3         |
+-------------+----------+------+----------+



**This is the same as the DENSE_RANK function in SQL.**

###2.4 percent_rank Window Function

In [0]:
df.withColumn('percent_rank', percent_rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|Maria        |Finance   |3000  |0.0         |
|Scott        |Finance   |3300  |0.5         |
|Jen          |Finance   |3900  |1.0         |
|Kumar        |Marketing |2000  |0.0         |
|Jeff         |Marketing |3000  |1.0         |
|James        |Sales     |3000  |0.0         |
|James        |Sales     |3000  |0.0         |
|Robert       |Sales     |4100  |0.5         |
|Saif         |Sales     |4100  |0.5         |
|Michael      |Sales     |4600  |1.0         |
+-------------+----------+------+------------+



###This is the same as the PERCENT_RANK function in SQL.

---
###2.5 ntile Window Function



**ntile() window function returns the relative rank of result rows within a window partition. In below example we have used 2 as an argument to ntile hence it returns ranking between 2 values (1 and 2)**

In [0]:
"""ntile"""

df.withColumn("ntile", ntile(2).over(windowSpec)).show(truncate=False)

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|Maria        |Finance   |3000  |1    |
|Scott        |Finance   |3300  |1    |
|Jen          |Finance   |3900  |2    |
|Kumar        |Marketing |2000  |1    |
|Jeff         |Marketing |3000  |2    |
|James        |Sales     |3000  |1    |
|James        |Sales     |3000  |1    |
|Robert       |Sales     |4100  |1    |
|Saif         |Sales     |4100  |2    |
|Michael      |Sales     |4600  |2    |
+-------------+----------+------+-----+



**This is the same as the NTILE function in SQL.**

---


##3. PySpark Window Analytic functions


###3.1 cume_dist Window Function


**cume_dist() window function is used to get the cumulative distribution of values within a window partition.**

**This is the same as the DENSE_RANK function in SQL.**

In [0]:
"""cume_dist"""

df.withColumn("cume_dist", cume_dist().over(windowSpec)).show(truncate=False)

+-------------+----------+------+------------------+
|employee_name|department|salary|cume_dist         |
+-------------+----------+------+------------------+
|Maria        |Finance   |3000  |0.3333333333333333|
|Scott        |Finance   |3300  |0.6666666666666666|
|Jen          |Finance   |3900  |1.0               |
|Kumar        |Marketing |2000  |0.5               |
|Jeff         |Marketing |3000  |1.0               |
|James        |Sales     |3000  |0.4               |
|James        |Sales     |3000  |0.4               |
|Robert       |Sales     |4100  |0.8               |
|Saif         |Sales     |4100  |0.8               |
|Michael      |Sales     |4600  |1.0               |
+-------------+----------+------+------------------+



###3.2 lag Window Function


**This is the same as the LAG function in SQL.**

In [0]:
"""lag"""

df.withColumn("lag", lag("salary", 2).over(windowSpec)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|lag |
+-------------+----------+------+----+
|Maria        |Finance   |3000  |null|
|Scott        |Finance   |3300  |null|
|Jen          |Finance   |3900  |3000|
|Kumar        |Marketing |2000  |null|
|Jeff         |Marketing |3000  |null|
|James        |Sales     |3000  |null|
|James        |Sales     |3000  |null|
|Robert       |Sales     |4100  |3000|
|Saif         |Sales     |4100  |3000|
|Michael      |Sales     |4600  |4100|
+-------------+----------+------+----+



##3.3 lead Window Function


**This is the same as the LEAD function in SQL.**

In [0]:
"""lead"""

df.withColumn("lead", lead("salary", 2).over(windowSpec)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |3900|
|Scott        |Finance   |3300  |null|
|Jen          |Finance   |3900  |null|
|Kumar        |Marketing |2000  |null|
|Jeff         |Marketing |3000  |null|
|James        |Sales     |3000  |4100|
|James        |Sales     |3000  |4100|
|Robert       |Sales     |4100  |4600|
|Saif         |Sales     |4100  |null|
|Michael      |Sales     |4600  |null|
+-------------+----------+------+----+



##4. PySpark Window Aggregate Functions


**In this section, I will explain how to calculate sum, min, max for each department using PySpark SQL Aggregate window functions and WindowSpec. When working with Aggregate functions, we don’t need to use order by clause.**

In [0]:
windowSpecAgg = Window.partitionBy("department")
df.withColumn("row", row_number().over(windowSpec))\
.withColumn("avg", avg("salary").over(windowSpecAgg))\
.withColumn("sum", sum("salary").over(windowSpecAgg))\
.withColumn("min", min("salary").over(windowSpecAgg))\
.withColumn("max", max("salary").over(windowSpecAgg))\
.where(col("row")==1).select("department", "avg", "sum", "min", "max")\
.show(truncate=False)


+----------+------+-----+----+----+
|department|avg   |sum  |min |max |
+----------+------+-----+----+----+
|Finance   |3400.0|10200|3000|3900|
|Marketing |2500.0|5000 |2000|3000|
|Sales     |3760.0|18800|3000|4600|
+----------+------+-----+----+----+



In [0]:
df.withColumn("row", row_number().over(windowSpec))\
.withColumn("avg", avg("salary").over(windowSpecAgg))\
.show()

+-------------+----------+------+---+------+
|employee_name|department|salary|row|   avg|
+-------------+----------+------+---+------+
|        Maria|   Finance|  3000|  1|3400.0|
|        Scott|   Finance|  3300|  2|3400.0|
|          Jen|   Finance|  3900|  3|3400.0|
|        Kumar| Marketing|  2000|  1|2500.0|
|         Jeff| Marketing|  3000|  2|2500.0|
|        James|     Sales|  3000|  1|3760.0|
|        James|     Sales|  3000|  2|3760.0|
|       Robert|     Sales|  4100|  3|3760.0|
|         Saif|     Sales|  4100|  4|3760.0|
|      Michael|     Sales|  4600|  5|3760.0|
+-------------+----------+------+---+------+

